<a href="https://colab.research.google.com/github/fatemafaria142/Impact-of-Instruction-Tuning-in-Biomedical-Language-Processing-Using-Different-LLMs/blob/main/Biomedical_Language_Processing_using_Mistral_7B_Instruct_v0_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Packages**

In [1]:
!pip install accelerate peft bitsandbytes transformers trl datasets torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00


#### **Dataset Link:** https://huggingface.co/datasets/nlpie/Llama2-MedTuned-Instructions?row=0

In [2]:
from datasets import load_dataset

instruct_tune_dataset = load_dataset("nlpie/Llama2-MedTuned-Instructions")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/205048 [00:00<?, ? examples/s]

### **Dataset structure**
* The dataset contains three columns.

In [3]:
instruct_tune_dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 205048
    })
})

In [4]:
# Display information for 5 data points from the 'train' split
num_samples_to_show = 5
for i in range(num_samples_to_show):
    data = instruct_tune_dataset['train'][i]
    print(f"Data Point {i + 1}:")
    print("Instruction:", data['instruction'])
    print("Input:", data['input'])
    print("Output:", data['output'])
    print("\n-----------------------------\n")


Data Point 1:
Instruction: Your goal is to determine the relationship between the two provided clinical sentences and classify them into one of the following categories:
Contradiction: If the two sentences contradict each other.
Neutral: If the two sentences are unrelated to each other.
Entailment: If one of the sentences logically entails the other.
Input: Sentence 1: For his hypotension, autonomic testing confirmed orthostatic hypotension.
Sentence 2: the patient has orthostatic hypotension
Output: Entailment

-----------------------------

Data Point 2:
Instruction: In the provided text, your objective is to recognize and tag gene-related Named Entities using the BIO labeling scheme. Start by labeling the initial word of a gene-related phrase as B (Begin), and then mark the following words in the same phrase as I (Inner). Any words not constituting gene-related entities should receive an O label.
Input: The first product of ascorbate oxidation , the ascorbate free radical ( AFR ) , 

### **We will use just a small subset of the data for this training example**

In [5]:
instruct_tune_dataset["train"] = instruct_tune_dataset["train"].select(range(3500))
instruct_tune_dataset["test"] = instruct_tune_dataset["train"].select(range(300))

In [6]:
instruct_tune_dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 3500
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 300
    })
})

* Note that this time, the tokenizer has added the control tokens `[INST]` and `[/INST]` to indicate the start and end of user messages (but not assistant messages!). **Mistral-instruct was trained with these tokens.**
* In order to leverage instruction fine-tuning, your prompt should be surrounded by `[INST]` and `[/INST]` tokens. The very first instruction should begin with a begin of sentence id. The next instructions should not. The assistant generation will be ended by the end-of-sentence token id.

In [12]:
def create_prompt(sample):
    """
    Update the prompt template:
    Combine both the prompt and input into a single column.
    """
    bos_token = "<s>"
    eos_token = "</s>"
    full_prompt = ""
    full_prompt += bos_token
    full_prompt += " [INST] "
    full_prompt += sample['instruction']
    full_prompt += sample['input']
    full_prompt += " [/INST] "
    full_prompt += sample['output']
    full_prompt += eos_token

    return full_prompt

In [13]:
create_prompt(instruct_tune_dataset["train"][0])

'<s> [INST] Your goal is to determine the relationship between the two provided clinical sentences and classify them into one of the following categories:\nContradiction: If the two sentences contradict each other.\nNeutral: If the two sentences are unrelated to each other.\nEntailment: If one of the sentences logically entails the other.Sentence 1: For his hypotension, autonomic testing confirmed orthostatic hypotension.\nSentence 2: the patient has orthostatic hypotension [/INST] Entailment</s>'

In [14]:
create_prompt(instruct_tune_dataset["train"][1])

'<s> [INST] In the provided text, your objective is to recognize and tag gene-related Named Entities using the BIO labeling scheme. Start by labeling the initial word of a gene-related phrase as B (Begin), and then mark the following words in the same phrase as I (Inner). Any words not constituting gene-related entities should receive an O label.The first product of ascorbate oxidation , the ascorbate free radical ( AFR ) , acts in biological systems mainly as an oxidant , and through its role in the plasma membrane redox system exerts different effects on the cell . [/INST] The : O\nfirst : O\nproduct : O\nof : O\nascorbate : O\noxidation : O\n, : O\nthe : O\nascorbate : O\nfree : O\nradical : O\n( : O\nAFR : O\n) : O\n, : O\nacts : O\nin : O\nbiological : O\nsystems : O\nmainly : O\nas : O\nan : O\noxidant : O\n, : O\nand : O\nthrough : O\nits : O\nrole : O\nin : O\nthe : O\nplasma : O\nmembrane : O\nredox : O\nsystem : O\nexerts : O\ndifferent : O\neffects : O\non : O\nthe : O\ncell

In [15]:
create_prompt(instruct_tune_dataset["train"][2])

'<s> [INST] If you have expertise in healthcare, assist users by addressing their medical questions and concerns.i am having acute gas problem which i am feeling  and my bp has raised to 130/90, During  this time i have palpitation and i feel there is a missing pulse . Kindly guide me to overcome this problem.On the other hand if there is no gas in my stomach the aforesaid problems are not occuring. [/INST] Dear-thanks for using our service and will try to help you with my medical advice. You might be experiencing intestinal gas from food that you are eating as some vegetables like broccoli, cauliflower and that can make your heart rate faster too. However, the blood pressure is high, which can be the primary problem or secondary to other things like salt in your diet. I recommend you to watch what your eat, control salt, Chat Doctor.  That will help your blood pressure and the intestinal movements too. Avoid soda and caffeine too. I hope that my advice has been helpful.</s>'

In [16]:
create_prompt(instruct_tune_dataset["train"][3])

"<s> [INST] Being a doctor, your task is to answer the medical questions based on the patient's description.\nDetermine the relation between the two medical terms.###Question: What is the relation between Household Activity and Does not bath self (finding) ? [/INST] ###Answer: Does not bath self (finding) interprets: Household Activity</s>"

In [17]:
create_prompt(instruct_tune_dataset["train"][4])

"<s> [INST] In your role as a medical professional, address the user's medical questions and concerns.Hi I have the last few weeks had a sharp pain in my upper abdoman (centre) and a dull pain on the side, changing from left to right..just under my chest/ribs I also feel dizzy and like im a bit drunk... I have see a doctor who says im aneamic and also referred me for a gastroscopy but im really worried its somethign that can t be seen with this camera.. [/INST] Thank you. Upper abdominal pain with anemia indicate some disease of upper GIT.peptic ulcer disease is the most possibility. Have you any history of vomiting out of blood or passage of black tarry stool ? Gastroscopy is confirmatory for this. Serology for h.pylori may be done. If these are inconclusive then other in. Should be considered like USG of abdomen. Dizziness is due to anemia. You have to check He level also.</s>"

In [18]:
create_prompt(instruct_tune_dataset["train"][5])

'<s> [INST] As a healthcare expert, provide answers to medical inquiries based on the information given by the user.i am scheduled for shock wave treatment for kidney stones and blockage. I understand they have more than 1 type of lithrotipsy machines, the ones that use water and those that use padded cushions. Which ones are recommended  and why? Do they both use x-rays to guide them? I know some use ultrasound. Thank you [/INST] Endoscopic lithography refers to the visualization of a calculus in the urinary tract and the simultaneous application of energy to fragment the stone or stones into either extractable or passable pieces. Many calculi in the upper urinary tract are treated with extracorporeal shockwave lithography (ESL). However, for stones that are poor candidates for this modality, endoscopic therapy is indicated. Ureteroscopy is the most common means of visualizing an upper urinary tract calculus. In addition, percutaneous techniques (e.g., percutaneous entomology) can als

### **Initializing the Model**
* Load the model using a 4-bit configuration, employing double quantization, and set bfloat16 as the compute data type.

* Notably, we opt for the instruct-tuned model in this instance rather than the base model. It's worth mentioning that fine-tuning a base model necessitates a more substantial amount of data!

In [19]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )


* https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2

In [20]:
mode_id = "mistralai/Mistral-7B-Instruct-v0.2"

In [21]:
model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto", use_cache=False
    )

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [22]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

### **Let's example how well the model does at this task currently:**

In [23]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=1024, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0].replace(prompt, "")

In [25]:
# Use a predefined template for instructions
prompt = '''<s>[INST] Your goal is to determine the relationship between the two provided clinical sentences and classify them into one of the following categories:
Contradiction: If the two sentences contradict each other.
Neutral: If the two sentences are unrelated to each other.
Entailment: If one of the sentences logically entails the other. '''
prompt += '''Sentence 1: For his hypotension, autonomic testing confirmed orthostatic hypotension.
Sentence 2: the patient has orthostatic hypotension
 [/INST]'''
print(prompt)

<s>[INST] Your goal is to determine the relationship between the two provided clinical sentences and classify them into one of the following categories:
Contradiction: If the two sentences contradict each other.
Neutral: If the two sentences are unrelated to each other.
Entailment: If one of the sentences logically entails the other. Sentence 1: For his hypotension, autonomic testing confirmed orthostatic hypotension.
Sentence 2: the patient has orthostatic hypotension
 [/INST]


In [26]:
generate_response(prompt, model)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


'<s><s> [INST] Your goal is to determine the relationship between the two provided clinical sentences and classify them into one of the following categories:\nContradiction: If the two sentences contradict each other.\nNeutral: If the two sentences are unrelated to each other.\nEntailment: If one of the sentences logically entails the other. Sentence 1: For his hypotension, autonomic testing confirmed orthostatic hypotension.\nSentence 2: the patient has orthostatic hypotension\n [/INST] Based on the provided clinical sentences, the relationship would be classified as "Entailment".\n\nThe reason is that Sentence 1 states that "autonomic testing confirmed orthostatic hypotension", which means that the patient has been diagnosed with orthostatic hypotension through an autonomic test. Sentence 2 states that "the patient has orthostatic hypotension", which is a simpler and more direct statement that the patient has the condition. Thus, Sentence 2 entails Sentence 1.</s>'

### **Setting up the Training**
we will be using the `huggingface` and the `peft` library!

In [27]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")


* we need to prepare the model to be trained in 4bit so we will use the  **`prepare_model_for_kbit_training`** function from peft




In [28]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# **Training Hyperparameters**
The choice of hyperparameters is contingent upon the desired training duration. Pay special attention to the following key factors:

* `num_train_epochs/max_steps:` Dictates the number of iterations over the data. Exercise caution, as an excessive number may lead to overfitting!

* `learning_rate:` Governs the convergence speed of the model. Adjust this parameter judiciously for optimal results.

In [29]:
from transformers import TrainingArguments
output_model= "mistral_instruct_generation"
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=1,
        max_steps=200,
        fp16=True,
)


### **Setting up the trainer**

`max_seq_length`: Context window size


In [30]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_prompt, # this will aplly the create_prompt mapping to all training and test dataset
  args=training_arguments,
  train_dataset=instruct_tune_dataset["train"],
  eval_dataset=instruct_tune_dataset["test"]
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:304: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


### **Training starts here**

In [31]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,2.617000
20,1.987300
30,2.013900
40,1.736300
50,1.438000
60,1.591700
70,1.528600
80,1.552300
90,1.509100
100,1.511700


TrainOutput(global_step=200, training_loss=1.5362527799606323, metrics={'train_runtime': 1794.4046, 'train_samples_per_second': 0.446, 'train_steps_per_second': 0.111, 'total_flos': 1.74835334381568e+16, 'train_loss': 1.5362527799606323, 'epoch': 0.43})

### **Save the model**

In [32]:
trainer.save_model("mistral_instruct_generation")

In [33]:
merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:229: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [34]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=1024, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0]

### **Example No:1**

In [39]:
# Use a predefined template for instructions
prompt = '''[INST] Being a doctor, your task is to answer the medical questions based on the patient's description.
Determine the relation between the two medical terms. '''
prompt += '''Question: What is the relation between Household Activity and Does not bath self (finding) ? [/INST]'''
response = generate_response(prompt, model)
# Print the response with formatted output
print(response)

<s> [INST] Being a doctor, your task is to answer the medical questions based on the patient's description.
Determine the relation between the two medical terms. Question: What is the relation between Household Activity and Does not bath self (finding) ? [/INST] Answer: Household Activity is a contributing factor for Does not bath self (finding).</s>


### **Example No:2**

In [38]:
# Use a predefined template for instructions
prompt = '''[INST] In your role as a medical professional, address the user's medical questions and concerns. '''
prompt += '''Hi I have the last few weeks had a sharp pain in my upper abdoman (centre) and a dull pain on the side, changing from left to right..just under my chest/ribs I also feel dizzy and like im a bit drunk... I have see a doctor who says im aneamic and also referred me for a gastroscopy but im really worried its somethign that can t be seen with this camera..[/INST]'''
response = generate_response(prompt, model)
# Print the response with formatted output
print(response)

<s> [INST] In your role as a medical professional, address the user's medical questions and concerns. Hi I have the last few weeks had a sharp pain in my upper abdoman (centre) and a dull pain on the side, changing from left to right..just under my chest/ribs I also feel dizzy and like im a bit drunk... I have see a doctor who says im aneamic and also referred me for a gastroscopy but im really worried its somethign that can t be seen with this camera..[/INST] Hello. I'm an internist, not a gastroenterologist, but I'll try to provide you with some helpful advice. Based on your description, you must have undergone recent investigation. Anemic states may lead to feelings of weakness and dizziness, but are unlikely to cause severe abdominal pain. Moreover, these symptoms improve with treatment, which typically consists of administering iron supplements. If you continue experiencing abdominal pain after receiving iron therapy, you should ask your doctor to consider other causes, such as pa

### **Example No:3**

In [37]:
# Use a predefined template for instructions
prompt = '''[INST] If you have expertise in healthcare, assist users by addressing their medical questions and concerns. '''
prompt += '''i am having acute gas problem which i am feeling  and my bp has raised to 130/90, During  this time i have palpitation and i feel there is a missing pulse . Kindly guide me to overcome this problem.On the other hand if there is no gas in my stomach the aforesaid problems are not occuring. [/INST]'''
response = generate_response(prompt, model)
# Print the response with formatted output
print(response)

<s> [INST] If you have expertise in healthcare, assist users by addressing their medical questions and concerns. i am having acute gas problem which i am feeling  and my bp has raised to 130/90, During  this time i have palpitation and i feel there is a missing pulse . Kindly guide me to overcome this problem.On the other hand if there is no gas in my stomach the aforesaid problems are not occuring. [/INST] Hi, thanks for reaching out. Given your symptoms have a possibility of being related to gaseous distension. I am assuming you are not passing flatus regularly. Its important first thing you need to make sure is your bowel movements are regular. If this is not the case due to your gas problem I will suggest you to consult to your doctor for some medicines to help ease gas and bloating. Also a fiber rich diet like Rajgira (Amarnath), Vegetables, Fruits, whole grain can help this issue gradually. The pulse is not an issue related to acute gas problem as it seems. There are many reasons